In [1]:
print("Welcome to my EMR Notebook!")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1592775982056_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Welcome to my EMR Notebook!

# Spark подключен

In [2]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

# Инициализируем SparkSession
spark = SparkSession.builder \
    .appName('Dreamlone') \
    .config('spark.executor.memory', '*') \
    .config("spark.cores.max", "*") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Установка пакетов, так как они не предустановлены на данный момент

In [3]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("matplotlib")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/af/f3/683bf2547a3eaeec15b39cef86f61e921b3b187f250fcd2b5c5fb4386369/pandas-1.0.5-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/d9/aa/22af3c636d6b87e8d7f8f28c02da338e348ed08beff1e2b58843f0f62fc1/matplotlib-3.2.2-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/31/b9/6202dcae729998a0ade30e80ac00f616542ef445b088ec970d407dfd41c0/kiwisolver-1.2.0-cp37-cp37m-manylinux1_x86_64.whl

In [28]:
# Импорт библиотек
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
data = spark.read.parquet('s3://dreamlone-mieca/CropAndClimate.parquet')
# Посмотрим какие есть колонки
data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Precip_amount', 'Precip_days', 'Pressure_mean', 'Sweden_pressure', 'Ireland_pressure', 'Portugal_pressure', 'Greece_pressure', 'Temperature_max', 'Temperature_min', 'Temperature_SAT', 'Year', 'Wheat', 'Rice', 'Maize', 'Potatoes', 'Peas', 'Barley', 'Nitrogen', 'Tractors', 'Population', 'Entity', 'Land_used', 'Cereal_production', 'Cereal_yield', 'RAINFALL', 'PRESSURE', 'MAX_TMP', 'TMP', 'MIN_TMP', '__index_level_0__']

In [6]:
data.filter(data.Entity == 'France').filter(data.Year > '1976') \
.select('Year', 'Precip_amount', 'Precip_days', 'RAINFALL').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------+-----------+--------------------+
|Year|     Precip_amount|Precip_days|            RAINFALL|
+----+------------------+-----------+--------------------+
|1977|          516.4519|       87.0|[4.633825, 7.3651...|
|1978|500.17194000000006|       86.0|[1.9677131, 3.261...|
|1979|         492.63788|       85.0|[7.3747, 3.536502...|
|1980|          397.0147|       71.0|[0.122699395, 0.4...|
|1981|         433.96133|       75.0|[0.02617587, 0.30...|
|1982|         329.65518|       63.0|[0.2819274, 0.920...|
|1983|         434.99136|       78.0|[0.85904914, 7.87...|
|1984|414.64437999999996|       72.0|[0.36899278, 0.08...|
|1985|         412.23276|       72.0|[0.12740286, 0.38...|
|1986|         445.59018|       80.0|[0.053016353, 0.1...|
|1987|          347.6998|       69.0|[1.4422555, 0.411...|
|1988|          519.6017|       82.0|[7.0522566, 1.353...|
|1989|          323.0364|       57.0|[0.0, 1.2782462, ...|
|1990|         343.08313|       60.0|[0.2951943, 0.640..

In [7]:
germany = data.filter(data.Entity == 'Germany').filter(data.Year == '2010') \
.select('Year', 'MAX_TMP', 'TMP', 'MIN_TMP')
germany.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+--------------------+--------------------+
|Year|             MAX_TMP|                 TMP|             MIN_TMP|
+----+--------------------+--------------------+--------------------+
|2010|[2.3872309, 4.601...|[-0.18742949, 1.9...|[-2.4357595, -1.8...|
+----+--------------------+--------------------+--------------------+

## Linear regression

In [8]:
# Функция для оценки величины ошибки на данных для выбранного года и выбранной страны
# year            --- Год, для которого будет даваться прогноз заблаговременностью несколько месяцев
# country         --- Страна, для которой будет делаться прогноз
# crop            --- Название с/х культуры
# data            --- датасет с данными
# additional_info --- необходимо ли выводить информацию о точности модели на тренировочных данных и схему ПРЕДСКАЗАНИЕ | ДЕЙСТВИТЕЛЬНОЕ ЗНАЧЕНИЕ | ПРЕДИКТОРЫ
def linear_regression(year, country, crop, data, additional_info = False):
    
    # Обучать модель мы будем для выбранной страны 
    train_dataset = data.filter(data.Entity == country).filter(data.Year > '1976').filter(data.Year < year)
    test_dataset = data.filter(data.Entity == country).filter(data.Year == year)

    # Теперь составим столбец 'features' в котором массивом будет содержаться информация о предикторах
    feat_cols = ['Precip_amount', 'Temperature_SAT', 'Precip_days', 'Pressure_mean', 'Temperature_min', 'Year']
    vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol = 'features')
    train_data = vec_assembler.transform(train_dataset)
    test_data = vec_assembler.transform(test_dataset)

    # Нормализуем наши признаки
    scaler = StandardScaler(inputCol = 'features', outputCol = 'features_scaled', withStd = True, withMean = False)
    scalerModel = scaler.fit(train_data)
    scaled_train_data = scalerModel.transform(train_data)
    scaled_test_data = scalerModel.transform(test_data)

    # Обучаем линейную регрессию
    lr = LinearRegression(featuresCol = 'features_scaled', labelCol = crop, maxIter=10, regParam = 0.3, elasticNetParam = 0.8)
    lr_model = lr.fit(scaled_train_data)
    trainingSummary = lr_model.summary
    
    # Применение модели на тестовом наборе данных
    lr_predictions = lr_model.transform(scaled_test_data)
    
    if additional_info == True:
        print('Training sample')
        print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
        print("r2: %f" % trainingSummary.r2, '\n')
        lr_predictions.select("prediction", crop, "features").show(5)
    
    # Возвращается действительное значение + предсказанное значение
    prediction = lr_predictions.select("prediction").collect()[0][0]
    actual = lr_predictions.select(crop).collect()[0][0]
    print('Error : ', prediction - actual)
    return(prediction, actual)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Applying a linear regression algorithm

In [10]:
preds = []
reals = []
for j in ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']:
    prediction, actual = linear_regression(year = j, country = 'France', crop = 'Wheat', data = data)
    preds.append(prediction)
    reals.append(actual)

# Переводим в numpy массивы
reals = np.array(reals)
preds = np.array(preds)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Error :  -0.15086555007976088
Error :  -0.37354675730386955
Error :  0.06422882414714337
Error :  0.5478980247098679
Error :  -0.04806165409800389
Error :  -0.19628799790222562
Error :  -0.2339138775278755
Error :  -0.6088975096297338
Error :  1.9256139354170543
Error :  -0.060261295156978534
Error :  0.07586685549946814